In [1]:
print("okie")

okie


In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [3]:
PINECONE_API_KEY = "pcsk_3tVDRV_9vnx6F8411DpGEvCUgx9E7ogz3guPB8VugKZSgitCYHxzrDx13dXuSRb1LaskQC"
PINECONE_API_ENV = "us-east-1"

In [4]:
def loadpdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [5]:
extracted_data = loadpdf("data/")

In [6]:
#extracted_data

In [7]:
# create text chunks 
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("Length of my chunk:", len(text_chunks))

Length of my chunk: 39994


In [9]:
# text_chunks

In [11]:
from langchain_huggingface import HuggingFaceEmbeddings

def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_huggingface_embeddings()

In [12]:
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [18]:
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore

# Set environment variable before using langchain_pinecone
os.environ["PINECONE_API_KEY"] = "pcsk_3tVDRV_9vnx6F8411DpGEvCUgx9E7ogz3guPB8VugKZSgitCYHxzrDx13dXuSRb1LaskQC"
os.environ["PINECONE_ENVIRONMENT"] = "us-east-1-aws"  # REQUIRED

# Initialize Pinecone client
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

index_name = "medical-bot"

# Create index if not exists
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Now use LangChain PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name,
)

In [28]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":5})

In [32]:
retrieved_docs = retriever.invoke("HIV")

In [33]:
retrieved_docs

[Document(id='f890df60-32a3-43ff-b71d-f54b82631888', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 1999.0, 'page_label': '1970', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'data\\Medical book.pdf', 'total_pages': 4505.0}, page_content='Immunodeficiency'),
 Document(id='703d10d4-d146-4438-bd1e-07e0b4c96bf1', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 1998.0, 'page_label': '1969', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'data\\Medical book.pdf', 'total_pages': 4505.0}, page_content='Immunodeficiency'),
 Document(id='6e01a0f6-07a0-46aa-87c1-9a3955bf385c', metadata={'creationdate': '2006-10-16T20:19:33+02:00', 'creator': 'Adobe Acrobat 6.0', 'moddate': '2006-10-16T22:03:45+02:00', 'page': 119.0, 'page_label': '90', 'producer': 'PDFlib+PDI 6.0.3 (SunOS)', 'source': 'data\\Medical book.pdf', 'tota

In [35]:
from together import Together

# Join the retrieved documents into a single string
retrieved_text = "\n".join([doc.page_content for doc in retrieved_docs])
user_question = "how to prevent HIV?"

# Prepare the system prompt for the model
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say that you don't know. "
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    f"{retrieved_text}"
)

# Step 4: Send the prompt to the LLM model using Together API
client = Together(api_key="48524a1dbffe66fd732b2d490d59c15c47ec440f48a1ed0cc7ba4cbda750be64")
response = client.chat.completions.create(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_question}
    ]
)

# Step 5: Print the response from the model
# Adjust to access the correct attribute in the response
print("Response:", response.choices[0].message.content)

Response: To prevent HIV, it's essential to practice safe sex, use protection such as condoms, and avoid sharing needles or other equipment that may be contaminated with the virus. Additionally, getting tested regularly and knowing your partner's status can also help prevent transmission. Abstaining from high-risk behaviors and considering pre-exposure prophylaxis (PrEP) if you're at high risk can also be effective prevention methods.
